In [20]:
# imports for heap
from heapq import heapify, heappush, heappop, nlargest
import random
import pandas as pd
# Import the f_select function
from f_select import f_select
# Import the generate schedule function
from orTree import OrTreeScheduler
from constr import Constr


In [21]:
# class
class ScheduleProcessor:
    """
    Class to manage and process a collection of schedules using a minimum heap data structure.
    
    This class supports three main operations:
    1. Adding a random schedule if the number of schedules is below a threshold. The random schedule will be generated using an Or-Tree function written by Emily
    2. Deleting the worst schedule if the number of schedules exceeds a predefined limit.
    3. Selecting the best schedule using a custom selection process called fselect. Fselect will be genearted by Kourosh

    Author:
        Khadeeja Abbas
    """

    def __init__(self):
        """
        Initializes the ScheduleProcessor with an empty heap.
        """
        # Create an empty heap for schedules
        self.heap = []
        heapify(self.heap)  # Convert the list into a valid heap structure

    def assignNum(self, limitOfSchedules):
        """
        Determines which operation to perform based on the number of schedules in the heap.

        Parameters:
            limitOfSchedules (int): The maximum allowable number of schedules in the heap.

        Returns:
            int: Operation code:
                0 -> Add a new random schedule.
                1 -> Remove the worst schedule.
                2 -> Select the next best schedule.
        """
        if len(self.heap) <= 4:
            num = 0  # Random operation
        elif len(self.heap) > limitOfSchedules:
            num = 1  # Deletion operation
        else:
            num = 2  # fselect
        return num

    def fwert(self, num):
        """
        Executes one of the three operations based on the input operation code.

        Parameters:
            num (int): Operation code determined by the assignNum method:
                0 -> Add a random schedule.
                1 -> Remove the worst schedule.
                2 -> Perform a selection operation.
        """
        match num:
            case 0:  # Add a random schedule
                # Generate a new schedule.
                # Load CSV with the first column as the index
                df_gslots = pd.read_csv('test_game_slots.csv', index_col=0)
                df_pslots = pd.read_csv('test_practice_slots.csv', index_col=0)
                df_events = pd.read_csv('test_events.csv', index_col=0)

                game_slots = list(df_gslots.index)
                practice_slots = list(df_pslots.index)
                games = len(df_events[df_events.Type == 'G'].index)
                practices = len(df_events[df_events.Type == 'P'].index)
                env = None

                constraints = Constr(df_gslots, df_pslots, df_events)

                scheduler = OrTreeScheduler(game_slots, practice_slots, games, practices, constraints, env)

                newSchedule = scheduler.generate_schedule()
                # Add the new schedule to the heap.
                heappush(self.heap, newSchedule)
                heapify(self.heap)  # Convert the list into a valid heap structure

                # print(self.heap)
                return

            case 1:  # Remove the worst schedule
                # Remove the largest value in the heap (the worst schedule in a min heap).
                n_largest = nlargest(1, self.heap)[0]
                self.heap.remove(n_largest)  # Remove by value
                heapify(self.heap)  # Convert the list into a valid heap structure

                print(self.heap)

                return

            case 2:  # Select the best schedule
                # Call the selection function.
                f_select(self.heap)
                heapify(self.heap)  # Convert the list into a valid heap structure
                return

            case _:  # Default case: also perform selection
                f_select(self.heap)
                heapify(self.heap)  # Convert the list into a valid heap structure

                return

# we needa keep in mind that we can get repeat schedules and we don't want that!
    def processSchedules(self, limitOfSchedules):
        """
        Main function to manage schedules. Determines the operation to perform and executes it.

        Steps:
        1. Define the maximum allowable number of schedules.
        2. Determine the appropriate operation using assignNum.
        3. Execute the operation using fwert.
        """
        
        while(1):
            # Determine the operation to perform
            num = self.assignNum(limitOfSchedules)
            # Executes the determined operation ONCE
            self.fwert(num)
        

In [22]:
# Example usage
if __name__ == "__main__":
    # Create a ScheduleProcessor instance
    processor = ScheduleProcessor()
    
    # Exampel Maximum number of schedules allowed in the heap
    limitOfSchedules = 30
    
    # Process schedules
    processor.processSchedules(limitOfSchedules)

NameError: name 'fitness' is not defined